Name: Alexis Jordan
Project: A* for 8 Queens

In [1]:
import copy
import heapq
import numpy as np

In [2]:
class Node:
    
    """
    The node class. 
    To instantiate a node, it's required to pass the puzzle, the parent node, the path cost and the heuristic
    For better readability, I also created the appropriate getters and setters
    """
    
    def __init__(self, puzzle, parent, g, h):
        self.puzzle = puzzle
        self.parent = parent
        self.g = g
        self.h = h
        self.f = g + h
        
    def get_parent(self):
        return self.parent
    
    def get_g(self):
        return self.g
    
    def get_h(self):
        return self.h
    
    def set_h(self, h):
        self.h = h
        
    def get_f(self):
        return self.f
    
    def get_path(self):                              # will return the path of the from the root to this node
        node, path = self, []
        while node:
            path.append(node)
            node = node.parent
        return reversed(path)

In [3]:
def find_blank(puzzle):
    
    """
    Finding the blank in the puzzle
    """
    for i in range(0,3):
        for j in range(0, 3):
            if puzzle[i][j] == 0:
                return i, j

In [4]:
def get_neighbors(node):
    
    
    blank = 0                                              # tell the function what the blank is
    
    blank_row, blank_col = find_blank(node.puzzle)        # find the location of the blank node
    
    up = blank_row - 1                                    # calculate the appropriate indices
    down = blank_row + 1
    right = blank_col + 1
    left = blank_col - 1
    
    neighbor_states = {"U": [], "D": [], "L": [], "R": []}         # store the neighbors in a dictionary
    
    if(up >= 0):                                                   # if the move is appropriate, make it and take the next step
        valueUp = copy.deepcopy(node.puzzle)
        valueUp[blank_row][blank_col] = valueUp[up][blank_col]
        valueUp[up][blank_col] = 0
        neighbor_states["U"].append(valueUp)
    
    if(down <= 2):
        valueDown = copy.deepcopy(node.puzzle)
        valueDown[blank_row][blank_col] = valueDown[down][blank_col]
        valueDown[down][blank_col] = 0
        neighbor_states["D"].append(valueDown)
    
    if(left >= 0):
        valueLeft = copy.deepcopy(node.puzzle)
        valueLeft[blank_row][blank_col] = valueLeft[blank_row][left]
        valueLeft[blank_row][left] = 0
        neighbor_states["L"].append(valueLeft)
    
    if(right <= 2):
        valueRight = copy.deepcopy(node.puzzle)
        valueRight[blank_row][blank_col] = valueRight[blank_row][right]
        valueRight[blank_row][right] = 0
        neighbor_states["R"].append(valueRight)
    
    return neighbor_states
    

In [5]:
def open_list_check(node, open_list):
    
    """
    Check to see if the node is already in the open list
    """
    passed_puzzle = np.array(node.puzzle)                # for ease, convert to np array
    
    for i in open_list:
        heuristic, node = heapq.heappop(open_list)              # pop the node off for comparison
        list_puzzle = np.array(node.puzzle)
        heapq.heappush(open_list, (heuristic, node))            # push the node back on for later
        if (passed_puzzle == list_puzzle).all():
            return True
        else:
            return False

In [6]:
def calc_heuristic(puzzle, goal):
    
    """
    Chosen Heuristic: Number of Misplaced Tiles
    """
    
    h = 0
    goal = np.array(goal)             # convert to np array for ease
    
    for i in range(0, 3):
        for j in range(0, 3):
            if puzzle[i][j] != goal[i][j] and puzzle[i][j] != None:         # count number of misplaced tiles
                h += 1
    return h

In [7]:
def count_inversion(puzzle):
    
    """
    This function counts the number of inversions within the puzzle
    """
    count = 0
    empty_value = 0 
    for i in range(0, 9):                  # loop through the puzzle
        for j in range(i + 1, 9):
            if puzzle[i] != empty_value and puzzle[j] != empty_value and puzzle[i] > puzzle[j]:  # if the tile is not blank, and the tile on the right is larger
                count += 1  # increment the count
    return count

In [8]:
def is_solvable(puzzle):
    
    """
    We iterate through the puzzles itself calculate the the total number of inversions
    If this number is not even then we can say that the puzzle is not solvable
    """
    total_count = count_inversion([i for row in puzzle for i in row])    # count the number of inversions in each row
    return (total_count % 2 == 0)                                        # will return false if odd (not solvable)

In [9]:
def goal_test(node, goal):
    
    """
    Check to see if the two arrays are equivalent.
    Convert them to np arrays for this
    """
    
    node = np.array(node.puzzle)
    goal = np.array(goal)
    
    return (node == goal).all()

In [10]:
def expandNode(node, open_list, closed_list, goal):
    
    """
    Take in the open list, closed list, node and goal
    Generate neighbors
    Add promising neighbors to the open list if it is not already in the open/closed list
    """
    
    neighbor_states = get_neighbors(node)
    
    children = list(neighbor_states.values())
    
    for i in range(0, len(children)):           # loop through all of the children (puzzles)
        
        if len(children[i]) != 0:               # if the list holding the child is not empty
            child = np.array(children[i])
            child.shape = (3,3)                  # reshape the list to proper dimension
            heuristic = calc_heuristic(child, goal)           # calculate heuristic
            child_g = node.get_g() + 1                        # calculate path cost
            child_node = Node(puzzle=child, parent=node, g=child_g, h=heuristic)     # create a node for each child puzzle
            
            if child_node.get_h() == 0:                      # in this case, this is essentially the goal node. Add it to the open list so we can expand it
                heapq.heappush(open_list, (heuristic, child_node))
            
            if not open_list_check(child_node, open_list) or closed_list:         # check the lists to see if this is node is already there
                if child_node.get_h() < node.get_h():                            # if the child node has a better heuristic, add it to the list
                    heapq.heappush(open_list, (heuristic, child_node))
          
    return open_list, closed_list

In [11]:
def AStar(initial_state, goal):
    
    initial_state = np.array(initial_state)                        # changing dimensions for ease of use
    initial_state = np.resize(initial_state, (3,3))
    
    goal = np.array(goal)
    goal = np.resize(goal, (3,3))
    
    if not is_solvable(initial_state):                            # checking inversions for solvability
        print("oh no. this puzzle is not solvable")
        return
        
    root = Node(puzzle=initial_state, parent=None, g=0, h=calc_heuristic(initial_state, goal))  # create root node
    open_list = []  
    closed_list = []
    nodes_expanded = 0


    while True:
        heapq.heappush(open_list, (root.get_h(), root))           # start the open list off with the root
        heuristic, curr_node = heapq.heappop(open_list)           # unpack important info from list
        closed_list.append(curr_node)
        
        if goal_test(curr_node, goal):                            # check if this node is the goal
            print("yay! we found it! \n")
            path = curr_node.get_path()
            
            for i in path:                                        # if it is, print some helpful stuff
                print("step: ")
                print(f"{i.puzzle} \n")
                print(f"path cost: {i.get_g()} \n")
            print(f"total nodes expanded: {nodes_expanded}")   
            break
        else:                      # if not, generate children so we can check some more nodes
            open_list, closed_list = expandNode(curr_node, open_list, closed_list, goal)    
            nodes_expanded += 1

In [12]:
initial_state = [1,2,3,4,5,6,8,7,0]
goal_state = [1,2,3,4,5,6,7,8,0]

In [13]:
AStar(initial_state, goal_state)

oh no. this puzzle is not solvable


In [14]:
initial_state = [[1,2,3], [4, 0, 5], [7, 8, 6]]
goal_state = [[1,2,3], [4, 5, 6], [7, 8, 0]]

In [15]:
AStar(initial_state, goal_state)

yay! we found it! 

step: 
[[1 2 3]
 [4 0 5]
 [7 8 6]] 

path cost: 0 

step: 
[[1 2 3]
 [4 5 0]
 [7 8 6]] 

path cost: 1 

step: 
[[1 2 3]
 [4 5 6]
 [7 8 0]] 

path cost: 2 

total nodes expanded: 2


In [16]:
initial_state = [[1,0,3], [4, 2, 5], [7, 8, 6]]
goal_state = [[1,2,3], [4, 5, 6], [7, 8, 0]]

In [17]:
AStar(initial_state, goal_state)

yay! we found it! 

step: 
[[1 0 3]
 [4 2 5]
 [7 8 6]] 

path cost: 0 

step: 
[[1 2 3]
 [4 0 5]
 [7 8 6]] 

path cost: 1 

step: 
[[1 2 3]
 [4 5 0]
 [7 8 6]] 

path cost: 2 

step: 
[[1 2 3]
 [4 5 6]
 [7 8 0]] 

path cost: 3 

total nodes expanded: 3


In [18]:
initial_state = [[1,2,3], [4, 5, 0], [7, 8, 6]]
goal_state = [[1,2,3], [4, 5, 6], [7, 8, 0]]

In [19]:
AStar(initial_state, goal_state)

yay! we found it! 

step: 
[[1 2 3]
 [4 5 0]
 [7 8 6]] 

path cost: 0 

step: 
[[1 2 3]
 [4 5 6]
 [7 8 0]] 

path cost: 1 

total nodes expanded: 1


In [20]:
initial_state = [[1,2,3], [4, 5, 6], [7, 8, 0]]
goal_state = [[1,2,3], [4, 5, 6], [7, 8, 0]]

In [21]:
AStar(initial_state, goal_state)

yay! we found it! 

step: 
[[1 2 3]
 [4 5 6]
 [7 8 0]] 

path cost: 0 

total nodes expanded: 0


In [22]:
initial_state = [[1,2,3], [4, 5, 6], [8, 7, 0]]
goal_state = [[1,2,3], [4, 5, 6], [7, 8, 0]]

In [23]:
AStar(initial_state, goal_state)

oh no. this puzzle is not solvable
